In [4]:
import pandas as pd

# === 1. LOAD & CLEANING DATA ===
df = pd.read_csv(r'filmtv_movies.csv')

# Pilih kolom yang diperlukan
df_cleaned = df[['country', 'year', 'genre', 'critics_vote', 'title']].copy()

# Hapus data kosong
df_cleaned.dropna(inplace=True)

# Konversi rating ke float
df_cleaned['critics_vote'] = pd.to_numeric(df_cleaned['critics_vote'], errors='coerce')
df_cleaned.dropna(subset=['critics_vote'], inplace=True)

# Filter nilai-nilai yang diinginkan
allowed_countries = ['Japan', 'Italy', 'United States']
allowed_genres = ['Drama', 'Action', 'Romantic']
allowed_years = [2021, 2022, 2023]
allowed_ratings = [7.00, 7.50, 8.00]

df_filtered = df_cleaned[
    (df_cleaned['country'].isin(allowed_countries)) &
    (df_cleaned['genre'].isin(allowed_genres)) &
    (df_cleaned['year'].isin(allowed_years)) &
    (df_cleaned['critics_vote'].isin(allowed_ratings))
].copy()

# Format rating jadi string rapi
df_filtered['critics_vote'] = df_filtered['critics_vote'].apply(lambda x: f"{x:.2f}")
df_filtered['year'] = df_filtered['year'].astype(str)

# Lihat kombinasi data yang tersisa
print("Jumlah data setelah difilter:", len(df_filtered))
print(df_filtered[['country', 'genre', 'year', 'critics_vote', 'title']].value_counts().head(20))

Jumlah data setelah difilter: 26
country        genre   year  critics_vote  title                       
Italy          Drama   2021  7.00          Il mio corpo vi seppellirà      1
                                           Naufragi                        1
United States  Drama   2022  8.00          Soft & Quiet                    1
                                           Alice                           1
                             7.50          Resurrection                    1
                             7.00          Palm Trees and Power Lines      1
                       2021  8.00          Jockey                          1
                             7.50          Land                            1
                             7.00          The Tender Bar                  1
                                           The Humans                      1
                                           Palmer                          1
                                           Mass 

In [5]:
class TreeNode:
    def __init__(self, value):
        self.value = value
        self.children = {}
        self.movies = []

def build_movie_tree(df):
    root = TreeNode("root")
    for _, row in df.iterrows():
        country = row["country"]
        genre = row["genre"]
        year = str(row["year"])
        rating = row['critics_vote']
        title = row["title"]

        if country not in root.children:
            root.children[country] = TreeNode(country)
        country_node = root.children[country]

        if genre not in country_node.children:
            country_node.children[genre] = TreeNode(genre)
        genre_node = country_node.children[genre]

        if year not in genre_node.children:
            genre_node.children[year] = TreeNode(year)
        year_node = genre_node.children[year]

        if rating not in year_node.children:
            year_node.children[rating] = TreeNode(rating)
        rating_node = year_node.children[rating]

        rating_node.movies.append(title)
    return root


In [6]:
import tkinter as tk
from tkinter import ttk, messagebox

# === GUI DEFINISI ===
def create_gui_with_tree(tree_root):
    root = tk.Tk()
    root.title("🎬 Rekomendasi Film (Tree-Based)")
    root.geometry("500x450")

    tk.Label(root, text="Pilih Kriteria Film", font=("Helvetica", 14, "bold")).pack(pady=10)

    # Dropdowns
    frame = tk.Frame(root)
    frame.pack(pady=10)

    tk.Label(frame, text="Country").grid(row=0, column=0, padx=5, pady=5, sticky='e')
    country_cb = ttk.Combobox(frame, values=["Japan", "Italy", "United States"], state="readonly")
    country_cb.grid(row=0, column=1)

    tk.Label(frame, text="Genre").grid(row=1, column=0, padx=5, pady=5, sticky='e')
    genre_cb = ttk.Combobox(frame, values=["Drama", "Action", "Romantic"], state="readonly")
    genre_cb.grid(row=1, column=1)

    tk.Label(frame, text="Year").grid(row=2, column=0, padx=5, pady=5, sticky='e')
    year_cb = ttk.Combobox(frame, values=["2021", "2022", "2023"], state="readonly")
    year_cb.grid(row=2, column=1)

    tk.Label(frame, text="Rating").grid(row=3, column=0, padx=5, pady=5, sticky='e')
    rating_cb = ttk.Combobox(frame, values=["7.00", "7.50", "8.00"], state="readonly")
    rating_cb.grid(row=3, column=1)

    result_text = tk.Text(root, height=10, wrap="word")
    result_text.pack(pady=10, padx=10, fill="both", expand=True)

    def search_using_tree():
        result_text.delete("1.0", tk.END)
        keys = [country_cb.get(), genre_cb.get(), year_cb.get(), rating_cb.get()]

        node = tree_root
        for key in keys:
            if key in node.children:
                node = node.children[key]
            else:
                messagebox.showinfo("Tidak Ditemukan", "❌ Tidak ada film yang cocok dengan kriteria.")
                return

        if node.movies:
            result_text.insert(tk.END, f"🎥 Film Ditemukan: {len(node.movies)}\n\n")
            for i, title in enumerate(node.movies, 1):
                result_text.insert(tk.END, f"{i}. {title}\n")
        else:
            messagebox.showinfo("Tidak Ditemukan", "❌ Tidak ada film yang cocok dengan kriteria.")

    tk.Button(root, text="Cari Rekomendasi", command=search_using_tree, bg="blue", fg="white").pack(pady=5)
    root.mainloop()

# === BANGUN TREE DAN JALANKAN GUI ===
tree = build_movie_tree(df_filtered)
create_gui_with_tree(tree)

